# Deep Learning with Tensorflow

Classical machine learning relies on using statistics to determine relationships between features and labels, and can be very effective for creating predictive models. However, a massive growth in the availability of data coupled with advances in the computing technology required to process it has led to the emergence of new machine learning techniques that mimic the way the brain processes information in a structure called an artificial neural network.

## Creating a Neural Network with Tensorflow
Tensorflow is a framework for creating machine learning models, including deep neural networks (DNNs). In this example, we'll use it to create a simple neural network that classifies iris flowers into species based on measurements of their petals and sepals.

> The iris classification model is a very common machine learning example, and the iris dataset is often the basis for "hello world" sample code for a wide range of machine learning frameworks. In reality, you can solve this problem easily using classical machine learning techniques without the need for a deep learning model; but it's a useful, easy to understand dataset with which to demonstrate the principles of neural networks in this notebook.

### Exploring the Iris Dataset
Before we start using Tensorflow to create a model, let's examine the iris dataset. Since this is a commonly used sample dataset, it is built-in to the *scikit-learn* machine learning library, so we'll get it from there. As with any supervised learning problem, we'll then split the dataset into a set of records with which to train the model, and a smaller set with which to validate the trained model.

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()

# Split data 70%-30% into training set and test set
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.40, random_state=0)

print ('Training Set: %d, Test Set: %d \n' % (len(x_train), len(x_test)))
print("Sample of features and labels:")
print('(features: ',iris.feature_names, ')')

# Take a look at the first 25 training features and corresponding labels
for n in range(0,24):
    print(x_train[n], y_train[n], '(' + iris.target_names[y_train[n]] + ')')

The *features* are the measurements for each iris observation, and the *label* is a numeric value that indicates the species of iris that the observation represents (versicolor, virginica, or setosa).

### Import Tensorflow Libraries
Since we plan to use Tensorflow to create our iris classifier, we'll need to install and import the libraries we intend to use. Tensorflow is already installed in your environment, so we just need to import the libraries we need.

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from tensorflow.keras import optimizers

print("Libraries imported.")
print('Keras version:',keras.__version__)
print('TensorFlow version:',tensorflow.__version__)

### Prepare the Data for Tensorflow
We've already loaded our data and split it into training and validation datasets. However, we need to do some further data preparation so that our data will work correctly with Tensorflow. Specifically, we need to set the data type of our labels to 32-bit floating point numbers, and specify that the labels represent categorical classes rather than numeric values.

In [ ]:
# Set data types for float features
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Set data types for categorical labels
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)
print('Ready...')

### Define a Neural Network
Now we're ready to define our neural network. In this case, we'll create a network that consists of 3 fully-connected layers:
* An input layer that receives four input values (the iris features) and applies a *ReLU* activation function to produce ten outputs.
* A hidden layer that receives ten inputs and applies a *ReLU* activation function to produce another ten outputs.
* An output layer that uses a *SoftMax* activation function to generate three outputs (which represent the probabilities for the three iris species)

In [ ]:
# Define a classifier network
hl = 10 # Number of hidden layer nodes

model = Sequential()
model.add(Dense(hl, input_dim=4, activation='relu'))
model.add(Dense(hl, input_dim=hl, activation='relu'))
model.add(Dense(3, input_dim=3, activation='softmax'))

print(model.summary())

### Train the Model
To train the model, we need to repeatedly feed the training values forward through the network, use a loss function to calculate the loss, use an optimizer to backpropagate the weight and bias value adjustments, and validate the model using the test data we withheld.

To do this, we'll apply a Stochastic Gradient Descent optimizer to a categorical cross-entropy loss function iteratively over 100 epochs.

In [ ]:
#hyper-parameters for optimizer
learning_rate = 0.01
learning_momentum = 0.9
sgd = optimizers.SGD(lr=learning_rate, momentum = learning_momentum)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

# Train the model over 50 epochs using 10-observation batches and using the test holdout dataset for validation
num_epochs = 50
history = model.fit(x_train, y_train, epochs=num_epochs, batch_size=10, validation_data=(x_test, y_test))

### Review Training and Validation Loss
After training is complete, we can examine the loss metrics we recorded while training and validating the model. We're really looking for two things:
* The loss should reduce with each epoch, showing that the model is learning the right weights and biases to predict the correct labels.
* The training loss and validations loss should follow a similar trend, showing that the model is not overfitting to the training data.

Let's plot the loss metrics and see:

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

epoch_nums = range(1,num_epochs+1)
training_loss = history.history["loss"]
validation_loss = history.history["val_loss"]
plt.plot(epoch_nums, training_loss)
plt.plot(epoch_nums, validation_loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()

### View the Learned Weights and Biases
The trained model consists of the final weights and biases that were determined by the optimizer during training. Based on our network model we should expect the following values for each layer:
* Layer 1: There are four input values going to ten output nodes, so there should be 10 x 4 weights and 10 bias values.
* Layer 2: There are ten input values going to ten output nodes, so there should be 10 x 10 weights and 10 bias values.
* Layer 3: There are ten input values going to three output nodes, so there should be 10 x 3 weights and 3 bias values.

In [ ]:
for layer in model.layers:
    weights = layer.get_weights()[0]
    biases = layer.get_weights()[1]
    print('------------\nWeights:\n',weights,'\nBiases:\n', biases)

### Evaluate Model Performance
So, is the model any good? The raw accuracy reported from the validation data would seem to indicate that it predicts pretty well; but it's typically useful to dig a little deeper and compare the predictions for each possible class. A common way to visualize the performace of a classification model is to create a *confusion matrix* that shows a crosstab of correct and incorrect predictions for each class.

In [ ]:
# Tensorflow doesn't have a built-in confusion matrix metric, so we'll use SciKit-Learn
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline


class_probabilities = model.predict(x_test)
predictions = np.argmax(class_probabilities, axis=1)
true_labels = np.argmax(y_test, axis=1)

# Plot the confusion matrix
cm = confusion_matrix(true_labels, predictions)
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.colorbar()
tick_marks = np.arange(len(iris.target_names))
plt.xticks(tick_marks, iris.target_names, rotation=85)
plt.yticks(tick_marks, iris.target_names)
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.show()

The confusion matrix should show a strong diagonal line indicating that there are more correct than incorrect predictions for each class.

### Using the Model with New Data
Now that we have a model we believe is reasonably accurate, we can use it to predict the species of new iris observations:

In [ ]:
# Save the trained model
modelFileName = 'models/iris-classifier.h5'
model.save(modelFileName)

# Load the saved model when required
del model  # deletes the existing model variable
model = models.load_model(modelFileName) # loads the saved model

x_new = np.array([[6.6,3.2,5.8,2.4]])
print ('New sample: {}'.format(x_new))

class_probabilities = model.predict(x_new)
predictions = np.argmax(class_probabilities, axis=1)

print(iris.target_names[predictions][0])

## Learn More
This notebook was designed to help you understand the basic concepts and principles involved in deep neural networks, using a simple Tensorflow example. To learn more about Tensorflow, take a look at the <a href="https://www.tensorflow.org/" target="_blank">Tensorflow web site</a>.